## Collect the Repository and Install all Dependencies
This may take some time.

In [ ]:
!if [ -d "outreach" ]; then echo "Repo already cloned." && cd outreach && git pull && cd ..; else echo "Collecting Outreach Repository" && git clone https://github.com/openneuropet/outreach.git; fi
from os.path import basename
from os import getcwd
if basename(getcwd()) == 'Neurohackademy2022':
    pass
else:
    %cd outreach/Neurohackademy2022/

!pip install nipype jedi pypet2bids

# Install Deno
!curl -fsSL https://deno.land/install.sh | sh
!export DENO_INSTALL="/root/.deno"
!export PATH="$DENO_INSTALL/bin:$PATH"

## Download Phantom ZIP and Extract

In [ ]:
!if [ ! -f "PHANTOMS.zip" ]; then wget -O PHANTOMS.zip https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip; fi
# unzip quietly in either case
!unzip -q -o PHANTOMS.zip

## Install Tree

In [ ]:
import subprocess
check_for_tree = subprocess.run(['which', 'tree'], capture_output=True)
if check_for_tree.returncode == 0:
    pass
else:
    import platform
    operating_system = platform.system()
    if operating_system == 'Linux':
        subprocess.run("apt-get install tree -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install tree", shell=True)
    else:
        print("You're on your own windows user.")

## Take a quick look at the raw dicom and ecat data that we've unzipped into this project folder.

In [ ]:
!tree OpenNeuroPET-Phantoms/sourcedata --filelimit 15

## Check for dcm2niix and install if it's not present.

Additionally, we tell pypet2bids where the dcm2niix executable is located at, this is best practice on windows and any sort of virtual environment/notebook as the $PATH variable can be "wonky" in the later case.

In [ ]:
# check for dcm2niix
import platform
import os
from pathlib import Path

check_dcm2niix = subprocess.run("which dcm2niix", shell=True, capture_output=True)
if check_dcm2niix.returncode == 0:
    print('dcm2niix is installed')
    version = subprocess.run('dcm2niix --version', shell=True, capture_output=True).stdout.decode()
    print(version)
    # set dcm2niix path as this is running in an ipython notebook
    dcm2niix_path = subprocess.run("which dcm2niix", shell=True, capture_output=True)
    subprocess.run(f"dcm2niix4pet --set-dcm2niix-path {dcm2niix_path.stdout.decode()}", shell=True)
else:
    print('dcm2niix is not installed')
    operating_system = platform.system()

    if operating_system == 'Linux':
        dcm2niix_install_dir = Path("dcm2niix_install")
        print(f"dcm2niix_install_dir {dcm2niix_install_dir}")
        if not dcm2niix_install_dir.exists():
            os.mkdir(dcm2niix_install_dir)
        subprocess.run("curl -fLO https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20230411/dcm2niix_lnx.zip",
                         cwd=dcm2niix_install_dir,
                         shell=True)
        subprocess.run("unzip dcm2niix*.zip",
                         shell=True,
                         cwd=dcm2niix_install_dir)
        if str(dcm2niix_install_dir) not in os.environ.get('PATH', ''):
            os.environ['PATH'] += os.pathsep + str(dcm2niix_install_dir.resolve())
        # ensure it's on the path
        check_dcm2niix_on_path = subprocess.run('dcm2niix --version && which dcm2niix', shell=True, capture_output=True)
        check_dcm2niix_on_path.stdout.decode()
    elif operating_system == 'Darwin':
        subprocess.run("brew install dcm2niix", shell=True)
    else:
        print("You're on your own windows user.")

In [ ]:
# import the relevant Python packages
import numpy
import nibabel
import nipype
import matplotlib
import subprocess

## Setup the bids-validator
This is the easiest way to get the bids validator running on a Colab Notebook, from here on out it can be called with `bids-validator()`

In [1]:
import subprocess

def bids_validator(path_to_dataset='.'):
  validator = subprocess.run(f"/root/.deno/bin/deno run --allow-read --allow-env https://deno.land/x/bids_validator/bids-validator.ts {path_to_dataset}", shell=True, capture_output=True, text=True)
  print(validator.stdout)

bids_validator('')

## Examine and convert some dicoms obtained from our PHANTOMS.zip with dcm2niix4pet.

In [ ]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder

In [ ]:
!tree ./mynewfolder

In [ ]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder2 --kwargs TimeZero=ScanStart Manufacturer=Siemens ManufacturersModelName=Biograph InstitutionName="NIH Clinical Center" BodyPart=Phantom Units=Bq/mL TracerName=none TracerRadionuclide=F18 InjectedRadioactivity=81.24 SpecificRadioactivity=13019.23 ModeOfAdministration=infusion FrameTimesStart=0 AcquisitionMode="list mode" ImageDecayCorrected=true ImageDecayCorrectionTime=0 AttenuationCorrection=MR-corrected FrameDuration=300 FrameTimesStart=0

## run only if freesurfer is installed

In [ ]:
!mkdir PETprocessing

In [ ]:
!cd PETprocessing

In [ ]:
!if [ -d "PET_pipelines" ]; then pushd PET_pipelines && git pull; else git clone https://github.com/openneuropet/PET_pipelines.git; fi

In [ ]:
!ls PET_pipelines/pyPetSurfer

In [ ]:
%cd PET_pipelines/pyPetSurfer
!wget /dev/null https://www.dropbox.com/sh/69dwtnv29wd7jlx/AADnw5FvAANpvzKAxVQTnyhBa?dl=0
!if [ -f AADnw5FvAANpvzKAxVQTnyhBa?dl=0 ]; then unzip -o AADnw5FvAANpvzKAxVQTnyhBa?dl=0; fi
!if [ ! -f AADnw5FvAANpvzKAxVQTnyhBa?dl=0 ]; then chmod +x ds001421-1.4.1.sh && mkdir ds001421-download/; fi
!if [ ! -f AADnw5FvAANpvzKAxVQTnyhBa?dl=0 ]; then cp ds001421-1.4.1.sh ds001421-download/; fi
!if [ ! -f AADnw5FvAANpvzKAxVQTnyhBa?dl=0 ]; then cd ds001421-download/ && ./ds001421-1.4.1.sh && rm ds001421-1.4.1.sh; fi

In [ ]:
!pip install mri_coreg

In [ ]:
!ls && python3 example.py

In [ ]:
!ls .